# Traditional methods for classification - SVM and KNN


In [ ]:
# Download TorchVision repo to use some files from
# references/detection
!pip install pycocotools --quiet
!git clone https://github.com/pytorch/vision.git
!git checkout v0.3.0

!cp vision/references/detection/utils.py ./
!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./

fatal: destination path 'vision' already exists and is not an empty directory.
fatal: not a git repository (or any of the parent directories): .git


Lets import the libraries

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import cv2
from xml.etree import ElementTree as et
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torch
import torchvision
from torchvision import transforms as torchtrans  
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from engine import train_one_epoch, evaluate
import utils
import transforms as T
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
files_dir = '../content/drive/MyDrive/train'
test_dir = '../content/drive/MyDrive/test'

class FruitImagesDataset(torch.utils.data.Dataset):

    def __init__(self, files_dir, width, height, transforms=None):
        self.transforms = transforms
        self.files_dir = files_dir
        self.height = height
        self.width = width
        
        self.imgs = [image for image in sorted(os.listdir(files_dir))
                        if image[-4:]=='.jpg']
                
        self.classes = [_, 'apple', 'banana', 'orange']

    def __getitem__(self, idx):

        img_name = self.imgs[idx]
        image_path = os.path.join(self.files_dir, img_name)

        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img_res = cv2.resize(img_rgb, (self.width, self.height), cv2.INTER_AREA)
        img_res /= 255.0
        
        annot_filename = img_name[:-4] + '.xml'
        annot_file_path = os.path.join(self.files_dir, annot_filename)
        
        boxes = []
        labels = []
        tree = et.parse(annot_file_path)
        root = tree.getroot()
        
        wt = img.shape[1]
        ht = img.shape[0]
        
        for member in root.findall('object'):
            labels.append(self.classes.index(member.find('name').text))
            
            xmin = int(member.find('bndbox').find('xmin').text)
            xmax = int(member.find('bndbox').find('xmax').text)
            
            ymin = int(member.find('bndbox').find('ymin').text)
            ymax = int(member.find('bndbox').find('ymax').text)
            
            xmin_corr = (xmin/wt)*self.width
            xmax_corr = (xmax/wt)*self.width
            ymin_corr = (ymin/ht)*self.height
            ymax_corr = (ymax/ht)*self.height
            
            boxes.append([xmin_corr, ymin_corr, xmax_corr, ymax_corr])
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["area"] = area
        target["iscrowd"] = iscrowd
        image_id = torch.tensor([idx])
        target["image_id"] = image_id

        if self.transforms:
            
            sample = self.transforms(image = img_res,
                                     bboxes = target['boxes'],
                                     labels = labels)
            
            img_res = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])
            
        return img_res, target

    def __len__(self):
        return len(self.imgs)


dataset = FruitImagesDataset(files_dir, 224, 224)
dataset_test = FruitImagesDataset(test_dir, 224, 224)
print('length of dataset = ', len(dataset), '\n')

img, target = dataset[78]
print(img.shape, '\n',target)

length of dataset =  240 

(224, 224, 3) 
 {'boxes': tensor([[ 22.4000,  36.4903, 163.1000,  68.6452],
        [ 24.8500,  39.3806, 163.4500,  94.2968],
        [ 28.0000,  52.3871, 166.9500, 127.8968],
        [ 71.0500,  59.6129, 193.9000, 157.5226]]), 'labels': tensor([2, 2, 2, 2]), 'area': tensor([ 4524.1865,  7611.3750, 10492.0693, 12028.2041]), 'iscrowd': tensor([0, 0, 0, 0]), 'image_id': tensor([78])}


In [ ]:
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=10, shuffle=True, collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=10, shuffle=False, collate_fn=utils.collate_fn)

In [ ]:
from google.colab.patches import cv2_imshow

x_train = []
y_train = []
for i in range(len(dataset)):
    img, target = dataset[i]
    boxes = target['boxes'].cpu().numpy()
    labels = target['labels'].cpu().numpy()
    for b, l in zip(boxes, labels):
      b = [int(p) for p in b]
      cropped_img  = img[b[1]:b[3], b[0]:b[2]]
      y_train.append(l)
      cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)

      img_reshape = cv2.resize(cropped_img, (60,60))
      x_train.append(img_reshape)


In [ ]:
x_test = []
y_test = []
for i in range(len(test_dataset)):
    img, target = test_dataset[i]
    boxes = target['boxes'].cpu().numpy()
    labels = target['labels'].cpu().numpy()
    for b, l in zip(boxes, labels):
      b = [int(p) for p in b]

      cropped_img  = img[b[1]:b[3], b[0]:b[2]]
      y_test.append(l)
      cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)

      img_reshape = cv2.resize(cropped_img, (60,60))
      x_test.append(img_reshape)


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix

x_train_flat = [img.flatten() for img in x_train]
x_train_flat = np.array(x_train_flat)

svm = SVC().fit(x_train_flat, y_train)
pred = svm.predict(x_train_flat)

print(classification_report(y_train, pred))
print(accuracy_score(y_train, pred))

              precision    recall  f1-score   support

           1       0.94      0.94      0.94       156
           2       0.98      0.94      0.96       169
           3       0.90      0.94      0.92       140

    accuracy                           0.94       465
   macro avg       0.94      0.94      0.94       465
weighted avg       0.94      0.94      0.94       465

0.9419354838709677


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

x_test_flat = [img.flatten() for img in x_test]
x_test_flat = np.array(x_test_flat)

pred = svm.predict(x_test_flat)

print(classification_report(y_test, pred))
print(accuracy_score(y_test, pred))

              precision    recall  f1-score   support

           1       0.83      0.83      0.83        35
           2       0.95      0.88      0.91        40
           3       0.82      0.88      0.85        42

    accuracy                           0.86       117
   macro avg       0.87      0.86      0.86       117
weighted avg       0.87      0.86      0.86       117

0.8632478632478633


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=4)

knn.fit(x_train_flat, y_train)

pred = svm.predict(x_train_flat)

print(classification_report(y_train, pred))
print(accuracy_score(y_train, pred))

              precision    recall  f1-score   support

           1       0.94      0.94      0.94       156
           2       0.98      0.94      0.96       169
           3       0.90      0.94      0.92       140

    accuracy                           0.94       465
   macro avg       0.94      0.94      0.94       465
weighted avg       0.94      0.94      0.94       465

0.9419354838709677


In [ ]:
y_pred = knn.predict(x_test_flat)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.81      0.74      0.78        35
           2       0.90      0.88      0.89        40
           3       0.78      0.86      0.82        42

    accuracy                           0.83       117
   macro avg       0.83      0.83      0.83       117
weighted avg       0.83      0.83      0.83       117

0.8290598290598291
